## PART 4: Main Func: with Modifier.py
Error: the NaN values for X show up when Modifier can't generate (given the interval) new values.

In [1]:
# IMPORT LIBRARIES
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
from itertools import compress

# import global parameters 
# TODO: make interface (widget) to load/change the parameters (low-priority)
from global_settings import mdv,vlv
from global_settings import mdv,simexSettings

#import classes for Modifier, Simulator, and Validator,
from Validator import Validator
from Modifier import Modifier
from Simulator import Simulator
from Logger import Logger

validator = Validator()
modifier = Modifier
simulator = Simulator
logger = Logger()

logger_main_arguments = {}
mainfunc = False

logger_main_arguments["log_contex"] = "overall MAIN stats"
logger_main_arguments["main_status"] = "begin cycle"
logger.log_main(logger_main_arguments)

mainfunc = True
# Initialize interval list
ranges_list=[(mdv["domain_min_range"], mdv["domain_max_range"])]





while mainfunc:
    
    # Calls Modifier Controller
    mod_x_list= modifier.modifier_controller(intervals_list=ranges_list,local_modifier=modifier.local_modifier_A, do_plot=simexSettings["do_plot"])

    # breaks loop if iterations end by granularity reached
    if not mod_x_list: # FALSE IF ["modifier_data_point"] < mdv["modifier_incremental_unit"]:
        logger_main_arguments["log_contex"] = "overall MAIN stats"
        logger_main_arguments["main_status"] = "no generated points"
        logger.log_main(logger_main_arguments)
        break
    
    # Calls Simulator
    mod_x,sim_y_list = simulator.simulator_controller(mod_x_list,selected_function=simulator.sim_func_B)
    assert len(mod_x) == len(sim_y_list)
    
    
    # print('\n\n Modx: ',mod_x,'\nsymx: ',sim_y_list)
    
    # Calls Validator controller
    ranges_list = validator.validator_controller(mod_x_list=np.array(mod_x),sim_y_list=np.array(sim_y_list))
    # Updates interval_list to new range output from validator controller

    if not ranges_list:
        mainfunc = False
        logger_main_arguments["log_contex"] = "overall MAIN stats"
        logger_main_arguments["main_status"] = "no unfit intervals"
        logger.log_main(logger_main_arguments)
logger_main_arguments["log_contex"] = "Overall Stats"                          
logger_main_arguments["main_status"] = "end cycle"
logger.log_main(logger_main_arguments)


# TODO: make a list of history variable that are needed in validator: i-e- number of points total, number of good points, etc. 

TypeError: Modifier.modifier_controller() got an unexpected keyword argument 'ranges_list'